# Ejercicios Tarea 3 Curso CE IABG del módulo Big Data Aplicado
## Jaime Rabasco Ronda

# Paso 1. Instalación de Hadoop y preparación del entorno

####1.1 Descargamos Hadoop

In [1]:
!wget https://downloads.apache.org/hadoop/common/hadoop-3.3.1/hadoop-3.3.1.tar.gz

--2022-02-26 10:49:06--  https://downloads.apache.org/hadoop/common/hadoop-3.3.1/hadoop-3.3.1.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 135.181.214.104, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 605187279 (577M) [application/x-gzip]
Saving to: ‘hadoop-3.3.1.tar.gz’

hadoop-3.3.1.tar.gz 100%[===================>] 577.15M  30.1MB/s    in 20s     

2022-02-26 10:49:26 (29.4 MB/s) - ‘hadoop-3.3.1.tar.gz’ saved [605187279/605187279]



####1.2 Descompresión de la distribución de hadoop

In [2]:
!tar -xzf hadoop-3.3.1.tar.gz

####1.3 Copiar a /usr/local# 

In [3]:
#copy  hadoop file to user/local
!cp -r hadoop-3.3.1/ /usr/local/

####1.4 Configurar el Hadoop JAVA HOME
Hadoop requiere que se establezca la ruta de acceso a Java, ya sea como una variable de entorno o en el archivo de configuración de Hadoop.

In [4]:
import os
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["PATH"] = os.environ["PATH"] + ":" + "/usr/local/hadoop-3.3.1/bin"

#### 1.5 Comprobando Hadoop
Comprobamos que lo tenemos instalado y nos da información sobre las opciones y parámetros de ejecución que tiene hadoop

In [5]:
!hadoop version

Hadoop 3.3.1
Source code repository https://github.com/apache/hadoop.git -r a3b9c37a397ad4188041dd80621bdeefc46885f2
Compiled by ubuntu on 2021-06-15T05:13Z
Compiled with protoc 3.7.1
From source with checksum 88a4ddb2299aca054416d6b7f81ca55
This command was run using /usr/local/hadoop-3.3.1/share/hadoop/common/hadoop-common-3.3.1.jar


#Paso 2. Instalación de Flume

####2.1 Descargamos Flume

In [6]:
%%bash
wget https://dlcdn.apache.org/flume/1.9.0/apache-flume-1.9.0-bin.tar.gz
tar xzf apache-flume-1.9.0-bin.tar.gz
mv apache-flume-1.9.0-bin/ /usr/local/

--2022-02-26 10:54:08--  https://dlcdn.apache.org/flume/1.9.0/apache-flume-1.9.0-bin.tar.gz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67938106 (65M) [application/x-gzip]
Saving to: ‘apache-flume-1.9.0-bin.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 32.6M 2s
    50K .......... .......... .......... .......... ..........  0% 24.2M 2s
   100K .......... .......... .......... .......... ..........  0% 39.3M 2s
   150K .......... .......... .......... .......... ..........  0%  134M 2s
   200K .......... .......... .......... .......... ..........  0%  146M 1s
   250K .......... .......... .......... .......... ..........  0%  154M 1s
   300K .......... .......... .......... .......... ..........  0%  206M 1s
   350K .......... .......... .......... .......... ..........  0% 38.9M 1s
   

####2.2 Variables de entorno

In [7]:
os.environ["FLUME_HOME"] = "/usr/local/apache-flume-1.9.0-bin"
os.environ["PATH"] = os.environ["PATH"] + ":" + "/usr/local/apache-flume-1.9.0-bin/bin"
os.environ["JAVA_OPTS"]= "-Xms400m -Xmx3000m -Dcom.sun.management.jmxremote"

####2.3 Comprobando Flume

In [8]:
!flume-ng version

Flume 1.9.0
Source code repository: https://git-wip-us.apache.org/repos/asf/flume.git
Revision: d4fcab4f501d41597bc616921329a4339f73585e
Compiled by fszabo on Mon Dec 17 20:45:25 CET 2018
From source with checksum 35db629a3bda49d23e9b3690c80737f9


#Paso 3. Ejercicio 1. Ingesta de datos vía Flume
## Spooldir

### 3.1 Creando directorios

In [9]:
!mkdir -p logs
!mkdir -p copy-logs

## 3.2 Creamos el fichero de configuración del agente

In [14]:
%%writefile monitor_flume.conf
# Configuración del Agente para monitorización de un directorio

#Definición del agente: nombre y componentes
agente.sources  = s1  
agente.sinks    = k1  
agente.channels = c1  
   
#Configuración de propiedades Source
agente.sources.s1.type              = spooldir
agente.sources.s1.spoolDir          = /content/logs
agente.sources.s1.basenameHeader    = true
agente.sources.s1.basenameHeaderKey = fileName

# Configuración de propiedades del canal
agente.channels.c1.type    = memory
#agente.channels.c1.type = file
#agente.channels.c1.type = SPILLABLEMEMORY
#agente.channels.c1.memoryCapacity = 10000
#agente.channels.c1.overflowCapacity = 1000000
#agente.channels.c1.byteCapacity = 800000
#agente.channels.c1.checkpointDir = /content/flume/checkpoint
#agente.channels.c1.dataDirs = /content/flume/data

# Configuración de propiedades del sink
agente.sinks.k1.type = file_roll
agente.sinks.k1.sink.directory = /content/copy-logs

#Vinculación de source y sink al canal creado
agente.sources.s1.channels = c1
agente.sinks.k1.channel    = c1

Overwriting monitor_flume.conf


## 3.3 Descargamos el fichero csv de Kaggle

In [99]:
!wget https://raw.githubusercontent.com/jaimerabasco/CE_IABG_BigDataAplicado_Actividades/main/Actividad3/Height_of_Male_and_Female_by_Country_2022.csv

--2022-02-26 13:41:40--  https://raw.githubusercontent.com/jaimerabasco/CE_IABG_BigDataAplicado_Actividades/main/Actividad3/Height_of_Male_and_Female_by_Country_2022.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7631 (7.5K) [text/plain]
Saving to: ‘Height_of_Male_and_Female_by_Country_2022.csv’

Height_of_Male_and_ 100%[===================>]   7.45K  --.-KB/s    in 0s      

2022-02-26 13:41:40 (85.2 MB/s) - ‘Height_of_Male_and_Female_by_Country_2022.csv’ saved [7631/7631]



##3.4 Lanzamos el agente Flume

In [100]:
!flume-ng agent --conf ./ -f ./monitor_flume.conf -n agente -Dflume.root.logger=INFO -Xmx3000m

Info: Including Hadoop libraries found via (/usr/local/hadoop-3.3.1/bin/hadoop) for HDFS access
Info: Including Hive libraries found via () for Hive access
+ exec /usr/lib/jvm/java-11-openjdk-amd64/bin/java -Xmx20m -Dflume.root.logger=INFO -Xmx3000m -cp '/content:/usr/local/apache-flume-1.9.0-bin/lib/*:/usr/local/hadoop-3.3.1/etc/hadoop:/usr/local/hadoop-3.3.1/share/hadoop/common/lib/*:/usr/local/hadoop-3.3.1/share/hadoop/common/*:/usr/local/hadoop-3.3.1/share/hadoop/hdfs:/usr/local/hadoop-3.3.1/share/hadoop/hdfs/lib/*:/usr/local/hadoop-3.3.1/share/hadoop/hdfs/*:/usr/local/hadoop-3.3.1/share/hadoop/mapreduce/*:/usr/local/hadoop-3.3.1/share/hadoop/yarn:/usr/local/hadoop-3.3.1/share/hadoop/yarn/lib/*:/usr/local/hadoop-3.3.1/share/hadoop/yarn/*:/lib/*' -Djava.library.path=:/usr/local/hadoop-3.3.1/lib/native org.apache.flume.node.Application -f ./monitor_flume.conf -n agente
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/apache-flume-1.9.0-

1. Copiar el fichero csv descargado utilizando el manejador de archivos de la izquierda a logs.
2. Comprobar el log de flume
3. Comprobar el directorio copy-logs.
4. Flume se queda en ejecución continua no permitiendo ejecutar más cosas, por lo tanto ir a Entorno de Ejecución -> Interrumpir Ejecución

# Paso 4. Ejercicio 2. Acceso a los datos usando Spark-Hive

####4.1 Instalación e Importación de Datos en Hive utilizando PySpark

In [23]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
import findspark
findspark.init()

#### 4.2 Leer un fichero con spark (necesario para poder leer los ficheros que vamos a cargar)

In [24]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()

sc = spark.sparkContext


## 5.3 Leer el CSV en un Dataframe 
---
Observa la ruta del fichero creado en la carpeta copy-logs por flume. Copia la ruta y pegala en la lectura de fuente de spark


In [101]:
data = spark.read.csv('/content/copy-logs/1645882908305-1',inferSchema=True, header=True)

In [ ]:
rddL=sc.textFile("/content/copy-logs/1645882908305-1")
rddL.collect()

In [ ]:
from pyspark.sql import HiveContext
hiveCtx = HiveContext(sc)
ex1 = hiveCtx.read.csv("/content/copy-logs/1645882908305-1")
ex1.registerTempTable("Height_es")
results = hiveCtx.sql("SELECT * FROM Height_es").show()

## 5.4 Filtrado del archivo csv, quedándote únicamente con las columnas “Country”, "Male Height in Cm","Female Height in Cm" y guardarlo en otro fichero

In [105]:
dataF=data.select("Country_Name", "Male_Height_Cm","Female_Height_Cm")

In [106]:
dataF.write.csv("Heights_filtrados.csv")

In [107]:
rddL=sc.textFile("Heights_filtrados.csv")
rddL.collect()

['Netherlands,183.78,170.36',
 'Montenegro,183.3,169.96',
 'Estonia,182.79,168.66',
 'Bosnia and Herzegovina,182.47,167.47',
 'Iceland,182.1,168.91',
 'Denmark,181.89,169.47',
 'Czech Republic,181.19,167.96',
 'Latvia,181.17,168.81',
 'Slovakia,181.02,167.12',
 'Slovenia,180.98,167.2',
 'Ukraine,180.98,166.62',
 'Croatia,180.76,166.8',
 'Serbia,180.74,168.29',
 'Lithuania,180.72,167.63',
 'Poland,180.69,165.78',
 'Finland,180.57,166.48',
 'Norway,180.48,166.45',
 'Sweden,180.46,166.67',
 'Germany,180.28,166.18',
 'Dominica,180.15,166.89',
 'Bermuda,179.72,166.11',
 'Puerto Rico,179.48,163.06',
 'Greece,179.26,165.81',
 'Belgium,179.09,163.4',
 'Ireland,179.04,164.5',
 'Lebanon,178.96,163.67',
 'Andorra,178.84,165.53',
 'Antigua and Barbuda,178.84,165.72',
 'Australia,178.77,164.67',
 'Canada,178.75,164.73',
 'Switzerland,178.73,164.33',
 'Grenada,178.7,165.99',
 'Belarus,178.69,166.93',
 'France,178.6,164.49',
 'Austria,178.52,166.93',
 'Luxembourg,178.46,165.07',
 'Cook Islands,178.32

## 5.5 Mostrando el nuevo fichero filtrado

In [108]:
from pyspark.sql import HiveContext
hiveCtx = HiveContext(sc)
ex1 = hiveCtx.read.csv("Heights_filtrados.csv")
ex1.registerTempTable("Height_es")
results = hiveCtx.sql("SELECT * FROM Height_es").show()

/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:604: FutureWarning: HiveContext is deprecated in Spark 2.0.0. Please use SparkSession.builder.enableHiveSupport().getOrCreate() instead.
  FutureWarning


+--------------------+------+------+
|                 _c0|   _c1|   _c2|
+--------------------+------+------+
|         Netherlands|183.78|170.36|
|          Montenegro| 183.3|169.96|
|             Estonia|182.79|168.66|
|Bosnia and Herzeg...|182.47|167.47|
|             Iceland| 182.1|168.91|
|             Denmark|181.89|169.47|
|      Czech Republic|181.19|167.96|
|              Latvia|181.17|168.81|
|            Slovakia|181.02|167.12|
|            Slovenia|180.98| 167.2|
|             Ukraine|180.98|166.62|
|             Croatia|180.76| 166.8|
|              Serbia|180.74|168.29|
|           Lithuania|180.72|167.63|
|              Poland|180.69|165.78|
|             Finland|180.57|166.48|
|              Norway|180.48|166.45|
|              Sweden|180.46|166.67|
|             Germany|180.28|166.18|
|            Dominica|180.15|166.89|
+--------------------+------+------+
only showing top 20 rows



/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


# Paso 6. Ejercicio 3 Limpieza de datos con PIG

## 6.1 Instalación de Pig

In [69]:
%%bash
wget https://downloads.apache.org/pig/pig-0.17.0/pig-0.17.0.tar.gz
tar -xzf pig-0.17.0.tar.gz
cp -r pig-0.17.0/ /usr/local/

--2022-02-26 13:04:42--  https://downloads.apache.org/pig/pig-0.17.0/pig-0.17.0.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 135.181.214.104, 2a01:4f8:10a:201a::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230606579 (220M) [application/x-gzip]
Saving to: ‘pig-0.17.0.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  273K 13m46s
    50K .......... .......... .......... .......... ..........  0%  544K 10m20s
   100K .......... .......... .......... .......... ..........  0% 55.9M 6m55s
   150K .......... .......... .......... .......... ..........  0% 78.8M 5m12s
   200K .......... .......... .......... .......... ..........  0%  275K 6m53s
   250K .......... .......... .......... .......... ..........  0%  142M 5m44s
   300K .......... .......... .......... .......... ..........  0% 56.5M 4m56s
   350K .......... ..........

## 6.2 Variables de Entorno

In [70]:
os.environ["PIG_HOME"] = "/usr/local/pig-0.17.0"
os.environ["PATH"] = os.environ["PATH"] + ":" + "/usr/local/pig-0.17.0/bin"
os.environ["PIG_CLASSPATH"] = "/usr/local/hadoop-3.3.0/conf"

## 6.3 Comprobación de instalación correcta

In [ ]:
!pig -h -version

## 6.4 Comprobamos el fichero creado en flume

Recuerda la ruta del fichero en copy-logs, como hicimos en hive-spark

In [109]:
!cat /content/copy-logs/1645882908305-1 | head

Rank,Country_Name,Male_Height_Cm,Female_Height_Cm,Male_Height_Ft,Female_Height_Ft
1,Netherlands,183.78,170.36,6.03,5.59
2,Montenegro,183.30,169.96,6.01,5.58
3,Estonia,182.79,168.66,6.00,5.53
4,Bosnia and Herzegovina,182.47,167.47,5.99,5.49
5,Iceland,182.10,168.91,5.97,5.54
6,Denmark,181.89,169.47,5.97,5.56
7,Czech Republic,181.19,167.96,5.94,5.51
8,Latvia,181.17,168.81,5.94,5.54
9,Slovakia,181.02,167.12,5.94,5.48


## 6.5 Nos aseguramos recoger las columnas correctamente

In [110]:
import pandas as pd
df = pd.read_csv("/content/copy-logs/1645882908305-1")
df.head(5)

,Rank,Country_Name,Male_Height_Cm,Female_Height_Cm,Male_Height_Ft,Female_Height_Ft
0,1,Netherlands,183.78,170.36,6.03,5.59
1,2,Montenegro,183.30,169.96,6.01,5.58
2,3,Estonia,182.79,168.66,6.00,5.53
3,4,Bosnia and Herzegovina,182.47,167.47,5.99,5.49
4,5,Iceland,182.10,168.91,5.97,5.54


In [111]:
df.columns

Index(['Rank', 'Country_Name', 'Male_Height_Cm', 'Female_Height_Cm',
       'Male_Height_Ft', 'Female_Height_Ft'],
      dtype='object')

## 6.6 Procesar los nulos

1. Comprobamos si hay nulos en nuestro fichero

In [112]:
print(df.shape)
pd.isnull(df).any()

(199, 6)


Rank                False
Country_Name        False
Male_Height_Cm      False
Female_Height_Cm    False
Male_Height_Ft      False
Female_Height_Ft    False
dtype: bool

2. Hacemos un copia del fichero original para posteriormente añadir valores nulos

In [114]:
!cp /content/copy-logs/1645882908305-1 /content/copy-logs/1645882908305-1_nulos

3. Abrimos el fichero creado y probocamos valores nulos y comprobamos de nuevo

In [115]:
import pandas as pd
df = pd.read_csv("/content/copy-logs/1645882908305-1_nulos")
df.head(5)

,Rank,Country_Name,Male_Height_Cm,Female_Height_Cm,Male_Height_Ft,Female_Height_Ft
0,1,Netherlands,183.78,170.36,6.03,5.59
1,2,Montenegro,183.30,169.96,6.01,5.58
2,3,Estonia,182.79,168.66,6.00,5.53
3,4,Bosnia and Herzegovina,182.47,167.47,5.99,5.49
4,5,Iceland,182.10,168.91,5.97,5.54


4. Comprobamos que ahora si detecta valores nulos

In [116]:
print(df.shape)
pd.isnull(df).any()

(199, 6)


Rank                False
Country_Name        False
Male_Height_Cm       True
Female_Height_Cm     True
Male_Height_Ft       True
Female_Height_Ft    False
dtype: bool

5. Proceso Pig para sustituir los valores nulos en los campos concretos. Almacenamos la salida procesada en /content/nulos

In [119]:
%%writefile procesar_nulos.pig 

height = load '/content/copy-logs/1645882908305-1_nulos' using PigStorage(',') as (
    Rank:int, Country_Name:chararray, Male_Height_Cm:float, Female_Height_Cm:float, Male_Height_Ft:float,
       Female_Height_Ft:float);
    
height_clean = foreach height generate Rank..Country_Name,
  ((Male_Height_Cm IS  NULL) ? 0 : Male_Height_Cm),((Female_Height_Cm IS  NULL) ? 0 : Female_Height_Cm),
  ((Male_Height_Ft IS  NULL) ? 0 : Male_Height_Ft) , Female_Height_Ft;
  

STORE height_clean INTO 'nulos' using PigStorage(',');

Overwriting procesar_nulos.pig


In [120]:
!pig procesar_nulos.pig

2022-02-26 13:49:28,471 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2022-02-26 13:49:28,472 INFO pig.ExecTypeProvider: Trying ExecType : MAPREDUCE
2022-02-26 13:49:28,472 INFO pig.ExecTypeProvider: Picked MAPREDUCE as the ExecType
2022-02-26 13:49:28,554 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2022-02-26 13:49:28,554 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1645883368542.log
2022-02-26 13:49:29,089 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2022-02-26 13:49:29,182 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2022-02-26 13:49:29,182 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:///
2022-02-26 13:49:29,222 [main] INFO  org.apache.pig.PigServer - Pig Script ID for the ses

6. Comprobación mediante Python/Pandas

In [123]:
df_clean = pd.read_csv("/content/nulos/part-m-00000", skiprows=1, header = None, names = df.columns)
print(df_clean.shape)
pd.isnull(df_clean).any()


(199, 6)


Rank                False
Country_Name        False
Male_Height_Cm      False
Female_Height_Cm    False
Male_Height_Ft      False
Female_Height_Ft    False
dtype: bool